In [21]:
from data_download import *
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
%matplotlib inline
pd.set_option('display.max_columns', None)

In [35]:
salespath = r'C:\Users\caell\flatiron\projects\phase_2_project\phase_2_project_chicago-sf-seattle-ds-082420\data\EXTR_RPSale.csv'
parcelpath = r'C:\Users\caell\flatiron\projects\phase_2_project\phase_2_project_chicago-sf-seattle-ds-082420\data\EXTR_Parcel.csv'
residentialpath = r'C:\Users\caell\flatiron\projects\phase_2_project\phase_2_project_chicago-sf-seattle-ds-082420\data\EXTR_ResBldg.csv'
sales = pd.read_csv(salespath, encoding = 'ISO-8859-1')
#parcels = pd.read_csv(parcelpath, encoding = 'ISO-8859-1')
#residences = pd.read_csv(residentialpath, encoding = 'ISO-8859-1')

C:\Users\caell\new_Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [36]:
sales = sales[sales['DocumentDate'].astype(str).str.endswith('2019')]
sales.head()


,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,PlatLot,PlatBlock,SellerName,BuyerName,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning
4,3027422,213043,120,12/20/2019,560000,20191226000848,,,,,,,DOYLE REGAN M+STERLING C ...,SHAW HEATHER C ...,3,6,3,N,N,N,N,1,8,
49,2999169,919715,200,07/08/2019,192000,20190712001080,,,,,,,WAGNERESTATES LLC ...,SCHAFFER CORBIN ...,3,2,3,N,N,N,N,1,3,
98,3000673,894444,200,06/26/2019,185000,20190722001395,,,,,,,MAY THOMAS A+SHIRLEY E ...,KOCHEL RICHARD J+JANICE M ...,3,2,3,N,N,N,N,1,3,
210,3002257,940652,630,07/22/2019,435000,20190730001339,,,,,,,CRAMER NEIL C+PAM R+ET AL ...,HANSON BRYAN L+KAILI ...,11,6,3,N,N,N,N,1,8,
420,3012934,615490,10,09/24/2019,775000,20190930002488,,,,,,,TALBOT MORGAN P ...,FICHTER CAROLINE ...,11,6,3,N,N,N,N,1,8,


In [43]:
combo = sales.merge(residences, on = ['Major','Minor'])
combo = combo.merge(parcels, on = ['Major','Minor'])
combo.head()

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,PlatLot_x,PlatBlock_x,SellerName,BuyerName,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning,BldgNbr,NbrLivingUnits,Address,BuildingNumber,Fraction,DirectionPrefix,StreetName,StreetType,DirectionSuffix,ZipCode,Stories,BldgGrade,BldgGradeVar,SqFt1stFloor,SqFtHalfFloor,SqFt2ndFloor,SqFtUpperFloor,SqFtUnfinFull,SqFtUnfinHalf,SqFtTotLiving,SqFtTotBasement,SqFtFinBasement,FinBasementGrade,SqFtGarageBasement,SqFtGarageAttached,DaylightBasement,SqFtOpenPorch,SqFtEnclosedPorch,SqFtDeck,HeatSystem,HeatSource,BrickStone,ViewUtilization,Bedrooms,BathHalfCount,Bath3qtrCount,BathFullCount,FpSingleStory,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost,PropName,PlatName,PlatLot_y,PlatBlock_y,Range,Township,Section,QuarterSection,PropType,Area,SubArea,SpecArea,SpecSubArea,DistrictName,LevyCode,CurrentZoning,HBUAsIfVacant,HBUAsImproved,PresentUse,SqFtLot,WaterSystem,SewerSystem,Access,Topography,StreetSurface,RestrictiveSzShape,InadequateParking,PcntUnusable,Unbuildable,MtRainier,Olympics,Cascades,Territorial,SeattleSkyline,PugetSound,LakeWashington,LakeSammamish,SmallLakeRiverCreek,OtherView,WfntLocation,WfntFootage,WfntBank,WfntPoorQuality,WfntRestrictedAccess,WfntAccessRights,WfntProximityInfluence,TidelandShoreland,LotDepthFactor,TrafficNoise,AirportNoise,PowerLines,OtherNuisances,NbrBldgSites,Contamination,DNRLease,AdjacentGolfFairway,AdjacentGreenbelt,HistoricSite,CurrentUseDesignation,NativeGrowthProtEsmt,Easements,OtherDesignation,DeedRestrictions,DevelopmentRightsPurch,CoalMineHazard,CriticalDrainage,ErosionHazard,LandfillBuffer,HundredYrFloodPlain,SeismicHazard,LandslideHazard,SteepSlopeHazard,Stream,Wetland,SpeciesOfConcern,SensitiveAreaTract,WaterProblems,TranspConcurrency,OtherProblems
0,3027422,213043,120,12/20/2019,560000,20191226000848,,,,,,,DOYLE REGAN M+STERLING C ...,SHAW HEATHER C ...,3,6,3,N,N,N,N,1,8,,1,1,27033 NE DOROTHY ST 98019,27033,,NE,DOROTHY,ST,,98019,1.0,7,0,1930,0,0,0,0,0,1930,0,0,0,0,550,,180,0,0,5,2,0,,3,0,0,2,1,0,0,0,1989,0,0,0,0,3,0,,DUVALL HILLS DIV NO. 04,12,,6,26,13,SE,R,70.0,2.0,NaN,NaN,DUVALL,1155,R4.5,1,1,2,10523,2,2,4,0,1,0,0,0,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,N,N,0,0,0,0,N,N,0,0,N,N,N,0,0,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N
1,3002257,940652,630,07/22/2019,435000,20190730001339,,,,,,,CRAMER NEIL C+PAM R+ET AL ...,HANSON BRYAN L+KAILI ...,11,6,3,N,N,N,N,1,8,,1,1,22848 SE 264TH CT 98038,22848,,SE,264TH,CT,,98038,2.0,7,0,1057,0,918,0,0,0,1975,0,0,0,0,465,,45,0,0,5,2,0,,3,0,2,1,0,1,0,0,1994,0,0,0,0,3,2500,,WILDERNESS ESTATES,63,,6,22,27,SW,R,56.0,3.0,NaN,NaN,MAPLE VALLEY,1813,R-6,1,1,2,7392,2,2,4,0,1,0,0,0,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,N,N,0,0,0,0,N,N,0,0,N,N,N,0,0,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N
2,3012934,615490,10,09/24/2019,775000,20190930002488,,,,,,,TALBOT MORGAN P ...,FICHTER CAROLINE ...,11,6,3,N,N,N,N,1,8,,1,1,10322 12TH AVE NW 98177,10322,,,12TH,AVE,NW,98177,1.0,7,0,1040,0,0,0,0,0,2080,1040,1040,7,0,0,N,0,0,0,5,1,100,N,4,0,1,1,1,0,0,0,1947,0,0,0,0,5,0,,NORTH VIEW TERRACE ADD,2,1,3,26,36,NW,R,39.0,3.0,NaN,NaN,SEATTLE,10,SF 7200,1,1,2,7102,2,2,4,0,1,0,2,0,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,N,N,0,0,0,0,N,N,0,0,N,N,N,0,0,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N
3,2987598,402770,820,04/15/2019,0,20190514000666,,,,,,,WONG TIRSA TABADA-PR ...,WONG TIRSA TABADA ...,3,6,15,N,N,N,N,18,8,12 18 31 51,1,1,19421 58TH AVE NE 98028,19421,,,58TH,AVE,NE,98028,1.0,7,0,1490,0,0,0,0,0,2930,1490,1440,7,0,0,Y,0,70,380,5,3,0,,3,0,1,2,0,1,0,1,1978,0,0,0,0,3,0,,LAKE FOREST PARK ESTATES ADD,31-32,5,4,26,2,SW,R,38.0,1.0,NaN,NaN,KENMORE,1442,R6,1,1,2,8003,2,2,3,0,1,0,2,0,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,N,N,0,0,0,0,N,N,0,0,N,N,N,0,0,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N
4,3030539,262404,9103,12/10/2019,0,20200121001491,,,,,,,JOHANSON ERIC A+WHITE 

In [38]:
combo = combo[(combo['SalePrice'] > 120000) & (combo['SalePrice'] < 5000000)]
combo.describe()

,ExciseTaxNbr,SalePrice,PropertyType,PrincipalUse,SaleInstrument,SaleReason,PropertyClass,BldgNbr,NbrLivingUnits,Stories,BldgGrade,BldgGradeVar,SqFt1stFloor,SqFtHalfFloor,SqFt2ndFloor,SqFtUpperFloor,SqFtUnfinFull,SqFtUnfinHalf,SqFtTotLiving,SqFtTotBasement,SqFtFinBasement,FinBasementGrade,SqFtGarageBasement,SqFtGarageAttached,SqFtOpenPorch,SqFtEnclosedPorch,SqFtDeck,HeatSystem,HeatSource,BrickStone,Bedrooms,BathHalfCount,Bath3qtrCount,BathFullCount,FpSingleStory,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost
count,2.857700e+04,2.857700e+04,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000,28577.000000
mean,2.999259e+06,7.945242e+05,8.137033,6.000875,3.845050,1.917101,7.975050,1.021801,1.017322,1.562778,7.729818,0.004339,1258.250166,44.648599,575.501802,16.375092,0.631627,0.530217,2188.216468,465.251426,294.728943,2.907198,82.705392,266.346852,80.343808,5.559471,132.066207,5.008083,2.111418,3.228365,3.464254,0.483186,0.538055,1.613045,0.636351,0.275326,0.062393,0.164188,1978.039332,98.303986,0.644294,0.137523,0.004759,3.426952,642.182139
std,1.671276e+04,5.214419e+05,5.903673,0.061755,3.810862,3.740304,0.231482,0.320505,0.166944,0.544585,1.155209,0.604014,504.324899,165.501165,690.294053,99.570393,20.361634,17.351383,973.623226,573.109135,444.499557,3.624408,182.103355,288.204685,129.325097,35.106853,210.742209,1.068961,0.573821,16.109494,0.963858,0.532081,0.659978,0.724971,0.625628,0.483191,0.250961,0.386178,32.130412,432.496104,6.748899,2.638808,0.420273,0.689663,2444.247763
min,2.968844e+06,1.205110e+05,0.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,2.985121e+06,4.750000e+05,3.000000,6.000000,3.000000,1.000000,8.000000,1.000000,1.000000,1.000000,7.000000,0.000000,940.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1470.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2.000000,0.000000,3.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1955.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
50%,2.999038e+06,6.600000e+05,11.000000,6.000000,3.000000,1.000000,8.000000,1.000000,1.000000,1.500000,8.000000,0.000000,1200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2020.000000,0.000000,0.000000,0.000000,0.000000,220.000000,40.000000,0.000000,0.000000,5.000000,2.000000,0.000000,3.000000,0.000000,0.000000,2.000000,1.000000,0.000000,0.000000,0.000000,1980.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
75%,3.013519e+06,9.200000e+05,11.000000,6.000000,3.000000,1.000000,8.000000,1.000000,1.000000,2.000000,8.000000,0.000000,1500.000000,0.000000,1140.000000,0.000000,0.000000,0.000000,2730.000000,930.000000,550.000000,7.000000,0.000000,480.000000,110.000000,0.000000,210.000000,5.000000,2.000000,0.000000,4.000000,1.000000,1.000000,2.000000,1.000000,1.000000,0.000000,0.000000,2006.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000
max,3.062899e+06,4.998000e+06,99.000000,10.000000,27.000000,19.000000,9.000000,16.000000,10.000000,3.500000,13.000000,99.000000,6923.000000,2210.000000,5110.000000,1805.000000,1310.000000,920.000000,10910.000000

In [ ]:
correlations = combo.corr()['SalePrice']

In [47]:
high_corrs = correlations.sort_values(ascending = False).head(20)
low_corrs = correlations.sort_values(ascending = True).head(20)

In [48]:
high_corrs

SalePrice             1.000000
BldgGrade             0.286463
SqFtTotLiving         0.271525
SqFt2ndFloor          0.219007
Stories               0.163624
SqFtOpenPorch         0.158357
LakeWashington        0.156037
BathFullCount         0.154760
SqFt1stFloor          0.144146
Township              0.133089
BathHalfCount         0.130067
Bath3qtrCount         0.119322
TidelandShoreland     0.115961
YrBuilt               0.114260
Bedrooms              0.114146
Territorial           0.107343
SqFtFinBasement       0.090397
SqFtGarageAttached    0.090039
FinBasementGrade      0.087968
BldgGradeVar          0.084039
Name: SalePrice, dtype: float64

In [49]:
low_corrs

SaleInstrument          -0.324062
SaleReason              -0.152966
PropertyClass           -0.075446
LevyCode                -0.027035
Condition               -0.026098
SqFtLot                 -0.020396
StreetSurface           -0.019095
PropertyType            -0.016208
TrafficNoise            -0.015394
Access                  -0.014594
HBUAsImproved           -0.014459
WfntFootage             -0.012281
HBUAsIfVacant           -0.011982
NbrLivingUnits          -0.011105
Area                    -0.009982
CurrentUseDesignation   -0.008767
Obsolescence            -0.006551
PcntNetCondition        -0.006262
HistoricSite            -0.005607
SqFtUnfinHalf           -0.004229
Name: SalePrice, dtype: float64

In [41]:
formula = 'SalePrice ~ SqFtTotLiving + BldgGrade'
model = smf.ols(formula = formula, data = combo).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.412
Model:                            OLS   Adj. R-squared:                  0.412
Method:                 Least Squares   F-statistic:                 1.001e+04
Date:                Mon, 28 Sep 2020   Prob (F-statistic):               0.00
Time:                        18:18:27   Log-Likelihood:            -4.0916e+05
No. Observations:               28577   AIC:                         8.183e+05
Df Residuals:                   28574   BIC:                         8.184e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -8.135e+05   1.85e+04    -44.063      0.000    -8.5e+05   -7.77e+05
SqFtTotLiving   184.5069      3.566     51.734      0.000     177.516     191.497
BldgGrade      1.558e+05   3005.835     51.833      0.000     1.5e+05    1.62e+05
==============================================================================
Omnibus:                    17182.281   Durbin-Watson:                   1.693
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           269237.452
Skew:                           2.614   Prob(JB):                         0.00
Kurtosis:                      17.099   Cond. No.                     1.89e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.89e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
residences['ZipCode'].value_counts()

98115         13588
98042         12064
98117         11087
98023         10815
98103         10683
              ...  
98107-5353        1
98075-9517        1
98059-7428        1
98025             1
988122            1
Name: ZipCode, Length: 298, dtype: int64

In [ ]:
resi_cols = ['SqFtTotLiving','BldgGrade','SqFtOpenPorch','SqFtEnclosedPorch','Bedrooms','YrBuilt']